In [58]:
import pandas as pd
import pandas_ta as ta
import numpy as np
import plotly.graph_objects as go
from scipy import stats

# Загрузка данных
df = pd.read_csv("./CSV/BTCUSDT/BTC_USDT_Last_365_Days_Candlestik_1h.csv")
# df = pd.read_csv("./CSV/EUR/EURUSD_Candlestick_1_Hour_BID_04.05.2003-15.04.2023.csv")
df = df[df['volume'] != 0]
df.reset_index(drop=True, inplace=True)

# Расчет EMA
df['EMA'] = ta.ema(df.close, length=150)
df = df[0:8000]

In [59]:
# Определение сигналов EMA
EMAsignal = [0] * len(df)
backcandles = 15

for row in range(backcandles, len(df)):
    upt = 1
    dnt = 1
    for i in range(row - backcandles, row + 1):
        if max(df.open[i], df.close[i]) >= df.EMA[i]:
            dnt = 0
        if min(df.open[i], df.close[i]) <= df.EMA[i]:
            upt = 0
    if upt == 1 and dnt == 1:
        EMAsignal[row] = 3
    elif upt == 1:
        EMAsignal[row] = 2
    elif dnt == 1:
        EMAsignal[row] = 1

df['EMASignal'] = EMAsignal

In [60]:
# Функция для определения pivot points
def isPivot(candle, window):
    if candle - window < 0 or candle + window >= len(df):
        return 0
    
    pivotHigh = 1
    pivotLow = 2
    for i in range(candle - window, candle + window + 1):
        if df.iloc[candle].low > df.iloc[i].low:
            pivotLow = 0
        if df.iloc[candle].high < df.iloc[i].high:
            pivotHigh = 0
    if pivotHigh and pivotLow:
        return 3
    elif pivotHigh:
        return pivotHigh
    elif pivotLow:
        return pivotLow
    else:
        return 0

In [61]:
window = 10
df['isPivot'] = df.apply(lambda x: isPivot(x.name, window), axis=1)

In [62]:
def pointpos(x):
    if x['isPivot'] == 2:
        return x['low'] - 1e-3
    elif x['isPivot'] == 1:
        return x['high'] + 1e-3
    else:
        return np.nan

In [63]:
df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)

In [64]:
# Функция для расчета уровней поддержки и сопротивления
def calculate_pivot_points(high, low, close):
    pivot_point = (high + low + close) / 3
    r1 = 2 * pivot_point - low
    r2 = pivot_point + (high - low)
    r3 = high + 2 * (pivot_point - low)
    s1 = 2 * pivot_point - high
    s2 = pivot_point - (high - low)
    s3 = low - 2 * (high - pivot_point)
    return pivot_point, r1, r2, r3, s1, s2, s3


In [65]:
# Вычисление уровней на основе выбранного периода
high = df['high'][7000:7200].max()
low = df['low'][7000:7200].min()
close = df['close'][7000:7200].iloc[-1]
pivot_point, r1, r2, r3, s1, s2, s3 = calculate_pivot_points(high, low, close)


In [66]:
# Отображение графика
dfpl = df[7000:7200]
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close'],
                name='Candlesticks')])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="Pivot Points")

# Добавление уровней поддержки и сопротивления на график
fig.add_hline(y=pivot_point, line=dict(color='blue', width=1), name='Pivot Point')
fig.add_hline(y=r1, line=dict(color='red', width=1), name='Resistance 1')
fig.add_hline(y=r2, line=dict(color='red', width=1, dash='dash'), name='Resistance 2')
fig.add_hline(y=r3, line=dict(color='red', width=1, dash='dot'), name='Resistance 3')
fig.add_hline(y=s1, line=dict(color='green', width=1), name='Support 1')
fig.add_hline(y=s2, line=dict(color='green', width=1, dash='dash'), name='Support 2')
fig.add_hline(y=s3, line=dict(color='green', width=1, dash='dot'), name='Support 3')

fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

KeyError: 'pivot_point'